In [1]:
import joblib
import numpy as np

from langchain.chat_models import ChatOpenAI
from discussion_agents.cog.agent.reflexion import ReflexionCoTAgent

In [2]:
hotpot = joblib.load('hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

hotpot['supporting_paragraphs'] = None
for ind, row in hotpot.iterrows():
    supporting_articles = row['supporting_facts']['title']
    articles = row['context']['title']
    sentences = row['context']['sentences'] 
    supporting_paragraphs = []
    for article in supporting_articles:
        supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
        supporting_paragraphs.append(supporting_paragraph)
    supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
    hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs

In [3]:
row = hotpot.iloc[0]

import dotenv
import os

dotenv.load_dotenv("../../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    temperature=0,
    max_tokens=250,
    model_name="gpt-3.5-turbo",
    model_kwargs={"stop": "\n"},
    openai_api_key=openai_api_key
)

agent = ReflexionCoTAgent(
    self_reflect_llm=llm,
    action_llm=llm,
    question=row['question'],
    context=row['supporting_paragraphs'],
    key=row['answer']
)

In [4]:
row["question"]

"VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"

In [5]:
row["answer"]

'Gesellschaft mit beschränkter Haftung'

In [6]:
row["supporting_paragraphs"]

'VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. It was founded for broadcast of VIVA Germany as VIVA Media AG in 1993 and has been owned by their original concurrent Viacom, the parent company of MTV, since 2004. Viva channels exist in some European countries; the first spin-offs were launched in Poland and Switzerland in 2000.\n\nA Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. In the United States, the equivalent type of entity is the limited liability company (LLC). The name of the GmbH form emphasizes the fact that the owners ("Gesellschafter", also known as members) of the entity are not personally liable for the company\'s debts. "GmbH"s are considered legal persons under German and Austrian law. Other variations include mbH (used when the term "Gesellschaft

: 

In [6]:
agents = [agent]
n = 1
for i in range(n):
    agent.generate(context=row["context"], question=row["question"], key=row["answer"], strategy=None)
    print(f'Answer: {agent.answer}')

Thought: The question is asking for the acronym that VIVA Media AG changed its name to in 2004. To find the answer, I need to look for information about VIVA Media AG's name change in the provided context. Let me search for that information.
Action: Finish[VIVA Media GmbH]
Answer: VIVA Media GmbH


In [8]:
row["answer"]

'Gesellschaft mit beschränkter Haftung'

: 

In [14]:
from discussion_agents.utils.parse import normalize_answer

In [15]:
sample_text = "A fox jumped over the fence. An apple was on the table. The quick brown fox."

result = normalize_answer(sample_text)
expected = " fox jumped over  fence.  apple was on  table.  quick brown fox."


In [16]:
result

'fox jumped over fence apple was on table quick brown fox'

: 

In [1]:
from discussion_agents.cog.agent.react import ReActAgent

In [2]:
import dotenv
import os

dotenv.load_dotenv("../../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_community.llms import OpenAI

agent = ReActAgent(llm=OpenAI(openai_api_key=openai_api_key))

In [4]:
agent

In [5]:
q = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

out = agent.generate(question=q)
print(out)


Thought: I need to find the name of the kick boxer who was once considered the best, but has been involved in controversies and crimes.
Action: Search[best kick boxer]
Observation 1: Jo Prestia (born 5 June 1960 in Porto Empedocle) is an Italian born French kick boxer and actor. He has appeared in more than seventy films since 1996 and is best known for his performance as Le Tenia in the controversial 2002 film, Irréversible.
Thought: Jo Prestia is a French kick boxer and actor, not the one I am looking for. I need to refine my search.
Action: Lookup[kick boxer]
Observation 2: (Result 1/1) Jo Prestia (born 5 June 1960 in Porto Empedocle) is an Italian born French kick boxer and actor. He has appeared in more than seventy films since 1996 and is best known for his performance as Le Tenia in the controversial 2002 film, Irréversible.
Thought: I need to add a keyword to my search to find the kick boxer I am looking for.
Action: Lookup[best kick boxer controversies]
Observation 3: No Resu